In [91]:
import pyspark

In [92]:
from pyspark.sql import SparkSession

In [93]:
spark = SparkSession.builder.appName('week5GraphTask2').enableHiveSupport().master('yarn').getOrCreate()

In [94]:
spark

In [95]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number,col,sum,count

In [96]:
data = spark.read.parquet("/data/sample264")

In [97]:
data.show()

+------+-------+--------+----------+
|userId|trackId|artistId| timestamp|
+------+-------+--------+----------+
| 13065| 944906|  978428|1501588527|
|101897| 799685|  989262|1501555608|
|215049| 871513|  988199|1501604269|
|309769| 857670|  987809|1501540265|
|397833| 903510|  994595|1501597615|
|501769| 818149|  994975|1501577955|
|601353| 958990|  973098|1501602467|
|710921| 916226|  972031|1501611582|
|  6743| 801006|  994339|1501584964|
|152407| 913509|  994334|1501571055|
|256855| 922936| 1001905|1501592059|
|366423| 858904|  974503|1501571685|
|479319| 816927|  975237|1501589184|
|589911| 960078|  995315|1501555145|
|673367| 825174|  970511|1501549783|
|  2237| 933052| 1000942|1501574276|
|108477| 850266|  974937|1501570638|
|225725| 878963|  986907|1501574389|
|302781| 934017|  974572|1501600590|
|399037| 901023|  974580|1501603463|
+------+-------+--------+----------+
only showing top 20 rows



In [98]:
datagrp = data.groupBy(col('userId'),col('artistId')).agg(count(col('userId')).alias('ct'))

In [99]:
window = Window.partitionBy(col('userId')).orderBy(col('ct').desc())

In [100]:
# datagrp.filter(col('userId') == 13065).show()

In [101]:
datalmt = datagrp.withColumn('row_number',row_number().over(window))\
    .filter(col('row_number')<= 1000).drop(col('row_number'))

In [90]:
# datalmt.show()

In [102]:
datalmtgrp = datalmt.groupBy(col('userId')).agg(sum(col('ct')).alias('sum_ct'))

In [103]:
data_norm  = datagrp.join(datalmtgrp,'userId','inner').withColumn('norm',col('ct')/col('sum_ct'))

In [105]:
res = data_norm.orderBy(col('norm').desc(),col('userId'),col('artistId')).limit(40)

In [106]:
res_final = res.select(col('userId'),col('artistId'))

In [107]:
for t1,t2 in res_final.collect():
    print('{}\t{}'.format(t1,t2))

66	993426
116	974937
128	1003021
131	983068
195	997265
215	991696
235	990642
288	1000564
300	1003362
321	986172
328	967986
333	1000416
346	982037
356	974846
374	1003167
428	993161
431	969340
445	970387
488	970525
542	969751
612	987351
617	970240
649	973851
658	973232
662	975279
698	995788
708	968848
746	972032
747	972032
776	997265
784	969853
806	995126
811	996436
837	989262
901	988199
923	977066
934	990860
957	991171
989	975339
999	968823
